In [1]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score, accuracy_score
import csv

In [2]:
def get_algo_path():
    import os
    cwd = os.getcwd()
    s='\\'
    if '\\' not in cwd:
        s='/'
    cwd= cwd.split(s)[4:-1]
    cwd.append('Not Normalized')
    cwd='/'.join(cwd)
    return cwd

def get_csv_path():
    import os
    cwd = os.getcwd()
    s="\\"
    if "\\" not in cwd:
        s='/'
    file= cwd.split(s)[:4]
    file.append('models_scores.csv')
    file= s.join(file)
    return file

def line_is_exist(file, row):
    logfile = open(file, 'r')
    loglist = logfile.readlines()
    logfile.close()
    for line in loglist:
        if ','.join(row) in line:
            return True
    return False

def write_new_score(file, line):
    if( not line_is_exist(file, line) ):
        with open(file, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(line)
    else:
        print('line exsist already')
    

In [3]:
data= pd.read_csv('../data.csv')

In [4]:
##copying data
data1=data.copy()
### spliting data en X et Y
X= data1.drop('Loan Status', axis=1)
Y= data1['Loan Status']
### spliting the data to train and test
X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 123)
## dicision tree (CART)
dct = DecisionTreeClassifier()
params={'criterion' : ['gini', 'entropy'],
        'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14, 15, 16, 17, None]}

##accuracy
grid_search_acc= GridSearchCV(estimator=dct, param_grid= params, scoring='accuracy',cv=2, n_jobs=-1)
grid_search_acc= grid_search_acc.fit(X_train, Y_train)
y_predict= grid_search_acc.best_estimator_.predict(X_test)


In [5]:
## get avg precision & avg recall
report= classification_report(Y_test, y_predict, output_dict=True)
avg_list = report.pop("weighted avg")
avg_precision= round(avg_list['precision'], 3)
avg_recall= round(avg_list['recall'], 3)
accuraccy= round(accuracy_score(Y_test,y_predict), 3)
## csv row
csv_row = [get_algo_path(), 'Decision Tree', str(grid_search_acc.best_params_), str(accuraccy), str(avg_precision), str(avg_recall)]
## write file
csv_file= get_csv_path()
write_new_score(csv_file, csv_row)

In [6]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_acc.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")


----------
best parameters:  {'criterion': 'entropy', 'max_depth': 12}
----------
[[13455  5905]
 [  793 18533]]
----------
              precision    recall  f1-score   support

           0       0.94      0.69      0.80     19360
           1       0.76      0.96      0.85     19326

    accuracy                           0.83     38686
   macro avg       0.85      0.83      0.82     38686
weighted avg       0.85      0.83      0.82     38686

----------


In [7]:
grid_search_acc.cv_results_



{'mean_fit_time': array([0.37300313, 0.43433976, 0.52310002, 0.60388625, 0.67569435,
        0.71957505, 0.77143764, 0.84424269, 0.76246059, 0.75547969,
        0.86319256, 0.8646872 , 0.91854358, 0.89311147, 1.12848115,
        0.50614488, 0.589921  , 0.68716264, 0.73702776, 0.8093363 ,
        0.87565875, 0.92452669, 1.03124201, 1.11102879, 1.21076214,
        1.29254234, 1.31648171, 1.46358812, 1.5074687 , 1.61717451]),
 'std_fit_time': array([0.00299323, 0.00149488, 0.01346481, 0.01346362, 0.03141534,
        0.01545918, 0.00748014, 0.01944745, 0.02443588, 0.0054878 ,
        0.00249314, 0.03191447, 0.07180643, 0.00448775, 0.06133628,
        0.02144325, 0.02543163, 0.03390956, 0.03091896, 0.04537785,
        0.05585027, 0.03091717, 0.0638293 , 0.06781876, 0.02792561,
        0.06881678, 0.06881726, 0.03839624, 0.05236006, 0.04338348]),
 'mean_score_time': array([0.02044523, 0.02094328, 0.02194226, 0.02144313, 0.02443337,
        0.02293885, 0.03191376, 0.02443469, 0.02343786, 0.02